In [1]:
import os
import sys
import platform

import torch
import torch.nn
import torchvision.datasets


from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.utils import save_image

In [2]:
print("Python Version: {}".format(platform.python_version()))
print("Torch Version: {}".format(torch.__version__))

Python Version: 3.7.1
Torch Version: 0.4.1


# Get MNIST Data

In [3]:
num_epochs = 11
batch_size = 128
imageSize = 28
underCompleteHiddenLayer = 64
overCompleteHiddenLayer = 1600
learning_rate = 1e-3

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [5]:
mnistTrainData = torchvision.datasets.MNIST("Data", download=True, train=True, transform=transform)
mnistTestData = torchvision.datasets.MNIST("Data", download=True, train=False, transform=transform)

In [6]:
print("Number of Training Samples: {}".format(len(mnistTrainData)))
print("Number of Testing Samples: {}".format(len(mnistTestData)))

Number of Training Samples: 60000
Number of Testing Samples: 10000


In [7]:
trainDataLoader = torch.utils.data.DataLoader(mnistTrainData, batch_size=batch_size, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(mnistTestData, batch_size=batch_size, shuffle=True)

# Autoencoder Model

In [8]:
class Autoencoder(torch.nn.Module):
    def __init__(self, hiddenLayer):
        super(Autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(torch.nn.Linear(28 * 28, hiddenLayer))
        self.decoder = torch.nn.Sequential(torch.nn.Linear(hiddenLayer, 28 * 28), torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train Model

In [9]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [10]:
def Train(epochs, hiddenLayer, trainDataLoader):
    model = Autoencoder(hiddenLayer)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(trainDataLoader):
            img = data.view(data.size(0), -1)
            img = Variable(img)

            output = model(img)
            loss = criterion(output, img)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.item()))
        if epoch % 10 == 0:
            pic = to_img(output.cpu().data)
            save_image(pic, 'image_{}_{}.png'.format(hiddenLayer, epoch))
    

In [11]:
Train(11, underCompleteHiddenLayer, trainDataLoader)

epoch [1/11], loss:0.0488
epoch [2/11], loss:0.0275
epoch [3/11], loss:0.0267
epoch [4/11], loss:0.0217
epoch [5/11], loss:0.0189
epoch [6/11], loss:0.0169
epoch [7/11], loss:0.0161
epoch [8/11], loss:0.0140
epoch [9/11], loss:0.0160
epoch [10/11], loss:0.0172
epoch [11/11], loss:0.0152


In [12]:
Train(11, overCompleteHiddenLayer, trainDataLoader)

epoch [1/11], loss:0.0076
epoch [2/11], loss:0.0055
epoch [3/11], loss:0.0052
epoch [4/11], loss:0.0044
epoch [5/11], loss:0.0047
epoch [6/11], loss:0.0046
epoch [7/11], loss:0.0047
epoch [8/11], loss:0.0049
epoch [9/11], loss:0.0048
epoch [10/11], loss:0.0048
epoch [11/11], loss:0.0054


# Visualize Ouput

In [13]:
# mnistTrainData[4][1]

In [14]:
# x = torch.nn.Tanh()(model.encoder(mnistTrainData[3][0].view(-1)))
# ?x = (model.encoder(mnistTrainData[4][0].view(-1))

In [15]:
# x = 0.5 * (x + 1)
# x = x.clamp(0, 1)
# x = x.view(1, 1, 20, 20)
# save_image(x, 'test.png'.format(epoch))

In [16]:
# x = model(mnistTrainData[3][0].view(-1))

In [17]:
# x = 0.5 * (x + 1)
# x = x.clamp(0, 1)
# x = x.view(1, 1, 28, 28)
# save_image(x, 'test2.png'.format(epoch))

# Visualize Latent Properties